In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from tensorflow.keras.layers import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
import joblib
import os

### 想辦法寫出文章轉成數字的方法 然後 寫成函式
### 找出讓分數更好一點的機器學習
### 自動化

In [79]:
def input_file(name):
    path = f"C:/Users/沈睿朋/Desktop/貼標/{name}" #資料夾目錄
    files= os.listdir(path) #得到資料夾下的所有檔名稱
    word = []
    word_text = ''
    for file in files: #遍歷資料夾
        if not os.path.isdir(file): #判斷是否是資料夾，不是資料夾才開啟
            f = open(path+"/"+file,encoding=('utf-8')); #開啟檔案
            iter_f = iter(f); #建立迭代器
        for i in iter_f:
            line = i.splitlines()
            if line == ['']:
                word.append(word_text)
                word_text = ''
            elif line == ['[廣告]請繼續往下閱讀...']:
                continue
            elif line[0][0] == 'h' or line[0][0] == '▲' or line[0][0:2] == '記者' or line[0][0] == '▼':
                continue
            word_text+=i.rstrip()
        word.append(word_text)
    return word , len(word)


domestic,d_num = input_file('國內')
foreign,f_num = input_file('國外')
data = pd.DataFrame(domestic+foreign,columns=['file'])
data['lable']= 1
data['lable'][d_num:] = 0 
print(len(data))

943


C:\Users\沈睿朋\AppData\Local\Temp/ipykernel_20140/2329507347.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['lable'][d_num:] = 0


In [80]:
import jieba
from jieba import enable_parallel
# jieba.enable_parallel() #并行分词开启
data['file_Participle'] = data['file'].apply(lambda i:jieba.cut(i) )
data['file_Participle'] =[' '.join(i) for i in data['file_Participle']]
data.head()

,file,lable,file_Participle
0,國人趁俄烏亂世逢低搶進！根據金管會最新統計，今（2022）年2月底，國內基金對俄曝險金額達2...,1,國人 趁 俄 烏亂 世逢 低 搶進 ！ 根據 金管會 最新 統計 ， 今 （ 2022 ） ...
1,美股週一漲跌互見，台股今 (1) 日開高走高，在鋼鐵、電動車概念股強漲帶領下，盤中漲近300...,1,美股 週一漲 跌 互見 ， 台股 今 ( 1 ) 日開 高 走高 ， 在 鋼鐵 、 ...
2,美股週一漲跌互見，台股今 （1）日開高走高，在鋼鐵、電動車概念股強漲帶領下，盤中漲近300點...,1,美股 週一漲 跌 互見 ， 台股 今 （ 1 ） 日開 高 走高 ， 在 鋼鐵 、 電動...
3,鋼鐵龍頭中鋼（2002）2021年獲利創新高，比市場預期高出2％，而中鋼擁有逾百萬名股東，預...,1,鋼鐵龍頭 中鋼 （ 2002 ） 2021 年 獲利創 新高 ， 比市場 預期 高出 2 ％...
4,美股週一漲跌互見，台股今 (1) 日開高走高，在鋼鐵、電動車概念股強漲帶領下，盤中漲近300...,1,美股 週一漲 跌 互見 ， 台股 今 ( 1 ) 日開 高 走高 ， 在 鋼鐵 、 ...


In [81]:
y = data['lable']
xtrain, xvalid, ytrain, yvalid = train_test_split(data.file_Participle.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)
print (xtrain.shape)
print (xvalid.shape)

(848,)
(95,)


In [82]:
# 利用刚才创建的NumberNormalizingVectorizer类来提取文本特征，注意里面各类参数的含义，自己去sklearn官方网站找教程看

stwlist=[line.strip() for line in open('C:/Users/沈睿朋/Desktop/貼標/stopwords0.txt',
'r',encoding='utf-8').readlines()]

ctv = CountVectorizer(min_df=3,
                      max_df=0.5,
                      ngram_range=(1,2),
                      stop_words = stwlist)

# 使用詞彙計數（Word Counts）作為功能，而不是使用TF-IDF。這可以使用scikit-learn中的CountVectorizer輕鬆完成。
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

# 基于tf-idf特征，使用xgboost
# clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
#                         subsample=0.8, nthread=10, learning_rate=0.1,eval_metric='mlogloss')
# clf.fit(xtrain_tfv.tocsc(), ytrain)
# predictions = clf.predict_proba(xvalid_tfv.tocsc())

# print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

# 基于word counts特征，使用xgboost
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1,eval_metric='mlogloss')
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['fed', 'jp', 'max', 'qe', 'vtfet'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


logloss: 0.324 


In [83]:
joblib.dump(clf,'clf.pkl')
joblib.dump(ctv,'ctv.pkl')

['ctv.pkl']